# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
file = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(file)
weather_df.head()

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
0,Busselton,AU,57.24,1613157827,89,0,11.54,-33.6500,115.3333
1,Lompoc,US,54.23,1613157828,62,20,17.27,34.6391,-120.4579
2,Nikolskoye,RU,0.68,1613157829,78,75,6.71,59.7035,30.7861
3,Adavere,EE,6.93,1613157830,85,90,3.44,58.7086,25.8994
4,Alofi,NU,78.80,1613157831,94,99,6.91,-19.0595,-169.9187


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the lat and lng columns as the locations variable
locations = weather_df[["Lat", "Lng"]]

# Use the humidity % column of the df as the weight variable
humidity = weather_df["Humidity %"]

In [24]:
# Generate the map
# From class discussion, it is important to set a center, and a zoom level to help make the map display properly
# Got the center and zoom_level values through trial and error
fig = gmaps.figure(center=(25.0, 15.0), zoom_level = 1.8)

# Generate a new layer on the map - Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# Per homework instructions, I will narrow down to cities with the following:
# Temp 80 > and < 70, Wind Speed < 10, Cloudiness = 0

# Start by dropping NA values
weather_df.dropna(inplace = True)

# Start by finding cities with the perfect temperature (Temp 80 > and < 70)
perfect_temperature = weather_df.loc[(weather_df["Temperature (F)"] < 80) & (weather_df["Temperature (F)"] > 70)]
perfect_temperature

# Next narrow it down further by looking for lower wind speeds (< 10)
less_wind = perfect_temperature.loc[(perfect_temperature["Wind Speed"] < 10)]
less_wind

# Finally remove all clouds in the sky by setting cloudiness equal to zero
# This creates our perfect weather dataframe
perfect_weather = less_wind.loc[(less_wind["Cloud"] == 0)]
perfect_weather

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# Create hotel_df based on the perfect weather df above
hotel_df = perfect_weather
hotel_df

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924


In [43]:
# Adding column "Hotel Name"
# Because I don't have the values for that column yet, I will set it equal to quotes to leave it blank (ignore pink message box)
hotel_df["Hotel Name"] = ""
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng,Hotel Name
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662,
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667,
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
